In [1]:
# Conexion a gdrive
from pydrive.auth import GoogleAuth
from google.colab import drive
from pydrive.drive import GoogleDrive
from google.colab import auth
import pandas as pd
from oauth2client.client import GoogleCredentials
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

nyra_2019_complete = '1AAshxhslPjhxg-UC86f9Z97s3W81nayF' #<-- Key del file en drive (nyra_2019_complete.csv 610 mb)
# nyra_race_table    = '1KtYQEvt4pboSqs7bO6jfdCHADo9PMzo9'#<-- Key del file en drive (nyra_race_table.csv 93 kb)
# nyra_start_table   = '1LYUx_De3U7TBw6GgqRk6whVzMjuJjSaz'#<-- Key del file en drive (nyra_start_table.csv 722 Kb)
# nyra_tracking_table= '15y4AtfLiyop9FWdc0SRaMUs-jKgnOppl'#<-- Key del file en drive (nyra_tracking_table.csv 320,7 mb)


download = drive.CreateFile({'id': nyra_2019_complete}) 
# Download the file to a local disc
download.GetContentFile('nyra_2019_complete.csv')
dtype_dict = {"track_id": object ,
              "race_date" :object, #will be read with "parse_dates" parameter
              "race_number" : np.int8, 
              "program_number" : object,
              "trakus_index" : np.int16,  
              "latitude" : np.float64,
              "longitude" : np.float64,
              "distance_id" : np.int16,  
              "course_type" : object, 
              "track_condition" : object,
              "run_up_distance" : np.int8,  
              "race_type" : object, 
              "purse" : np.int32,  
              "post_time" : np.int16,  
              "weight_carried" : np.int16,  
              "jockey" : object, 
              "odds" : np.int16,
              "position_at_finish":np.int8
             }
nyra_2019 = pd.read_csv('nyra_2019_complete.csv', header=None,dtype=dtype_dict,parse_dates=[1])
nyra_2019.columns= dtype_dict.keys()
nyra_2019_nn = nyra_2019.copy()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
positions = pd.DataFrame(
    nyra_2019[(nyra_2019["trakus_index"] == 1)]
    .groupby(["race_number", "race_date"])["track_id"]
    .count()
).reset_index()
positions = positions.rename(columns={"track_id": "participantes"})
nyra_2019_naive = nyra_2019[nyra_2019["trakus_index"] == 1].sort_values(
    by=["race_date", "track_id", "race_number", "odds"],
    ascending=True,
    na_position="first",
)

nyra_2019_naive = nyra_2019_naive.merge(
    positions, on=["race_number", "race_date"], how="left"
)

nyra_2019_naive["First_3"] = "Unknown"
nyra_2019_naive.loc[
    nyra_2019_naive["position_at_finish"].between(0, 4, inclusive=False), "First_3"
] = 1
nyra_2019_naive.loc[
    nyra_2019_naive["position_at_finish"].between(4, 60, inclusive=True), "First_3"
] = 0

nyra_2019_naive["Percentage_first_3"] = 3 / nyra_2019_naive["participantes"]

nyra_2019_naive["odds_ranking"] = nyra_2019_naive.groupby(["race_number", "race_date"])[
    "odds"
].rank(method="first", ascending=False)
nyra_2019_naive.sort_values(by=["race_date", "track_id", "race_number"])

## No se añade la distancia, ni el purse porque son constantes para todas las carreras y añaden ruido a la muestra, por eso tendía a una línea recta de predicción
df_ann_1 = nyra_2019_naive.copy()
df_ann_1 = df_ann_1.drop("jockey", axis=1)
test = (
    df_ann_1[["race_date", "track_id", "race_number"]]
    .sort_values(by=["race_date", "track_id", "race_number"])
    .drop_duplicates()
    .head(600)
)
train = (
    df_ann_1[["race_date", "track_id", "race_number"]]
    .sort_values(by=["race_date", "track_id", "race_number"])
    .drop_duplicates()
    .tail(1400)
)
train_total = (
    train.merge(df_ann_1, on=["race_date", "race_number", "track_id"], how="left")
    .sort_values(
        by=["race_date", "track_id", "race_number", "odds_ranking"],
        ascending=True,
        na_position="first",
    )
    .reset_index()
)
test_total = (
    test.merge(df_ann_1, on=["race_date", "race_number", "track_id"], how="left")
    .sort_values(
        by=["race_date", "track_id", "race_number", "odds_ranking"],
        ascending=True,
        na_position="first",
    )
    .reset_index()
)


df_ann_1 = nyra_2019_naive.copy()
df_ann_1 = df_ann_1.drop("jockey", axis=1)
train = train_total.drop("race_date", axis=1)
test = test_total.drop("race_date", axis=1)
TargetVariable = ["First_3"]
Predictors = ["weight_carried", "odds_ranking", "Percentage_first_3"]
X_train = train[Predictors]
y_train = train[TargetVariable]
X_test = test[Predictors]
y_test = test[TargetVariable]
X_train = np.asarray(X_train).astype(np.float32)
y_train = np.asarray(y_train).astype(np.float32)

from keras.models import Sequential
from keras.layers import Dense

classifier = Sequential()
# Defining the Input layer and FIRST hidden layer,both are same!
# relu means Rectifier linear unit function
classifier.add(
    Dense(units=5, input_dim=3, kernel_initializer="uniform", activation="sigmoid")
)

# Defining the SECOND hidden layer, here we have not defined input because it is
# second layer and it will get input as the output of first hidden layer
classifier.add(Dense(units=4, kernel_initializer="uniform", activation="sigmoid"))
classifier.add(Dense(units=1, kernel_initializer="uniform", activation="sigmoid"))

# Optimizer== the algorithm of SGG to keep updating weights
# loss== the loss function to measure the accuracy
# metrics== the way we will compare the accuracy after each step of SGD
classifier.compile(optimizer="adam", loss="binary_crossentropy")

# fitting the Neural Network on the training data
classifier.fit(X_train, y_train, batch_size=32, epochs=40, verbose=0)
dataframe = pd.DataFrame(classifier.predict(X_test).tolist())
dataframe = dataframe.rename(columns={0: "prediccion"})

carreras_test = test_total[
    [
        "race_date",
        "race_number",
        "position_at_finish",
        "First_3",
        "weight_carried",
        "odds",
        "track_id",
        "Percentage_first_3",
    ]
]
Base_prediccion = pd.concat([carreras_test, dataframe], axis=1).sort_values(
    by=["race_date", "track_id", "race_number", "odds", "position_at_finish"]
)
Base_prediccion["Prediccion_ranking"] = Base_prediccion.groupby(
    ["race_number", "race_date"]
)["prediccion"].rank(method="first", ascending=False)
Base_prediccion["First_3_ranking"] = np.where(
    Base_prediccion["Prediccion_ranking"].between(0, 5, inclusive=False),
    1,
    np.where(
        Base_prediccion["Prediccion_ranking"].between(4, 30, inclusive=False),
        0,
        "Unknown",
    ),
)

print(
    confusion_matrix(
        Base_prediccion[["First_3"]].astype(int),
        Base_prediccion[["First_3_ranking"]].astype(int),
    )
)
from sklearn import metrics

print(
    "Precision:",
    metrics.precision_score(
        Base_prediccion[["First_3"]].astype(int),
        Base_prediccion[["First_3_ranking"]].astype(int),
    ),
)
print(
    "Recall:",
    metrics.recall_score(
        Base_prediccion[["First_3"]].astype(int),
        Base_prediccion[["First_3_ranking"]].astype(int),
    ),
)
print(
    "Accuracy:",
    metrics.accuracy_score(
        Base_prediccion[["First_3"]].astype(int),
        Base_prediccion[["First_3_ranking"]].astype(int),
    ),
)
print(
    "F1_Score:",
    metrics.f1_score(
        Base_prediccion[["First_3"]].astype(int),
        Base_prediccion[["First_3_ranking"]].astype(int),
    ),
)

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score


y_pred = Base_prediccion[["First_3_ranking"]].astype(int)
y_test =Base_prediccion[["First_3"]].astype(int)
d1 = {'Modeblo': ['Red_Neuronal_1'], 
     'Precision': precision_score(y_test, y_pred.round()), 
     'Recall': recall_score(y_test, y_pred.round()),
     'Accuracy': accuracy_score(y_test, y_pred.round()),
      'F1_Score': f1_score(y_test, y_pred.round())}
Resultados1 = pd.DataFrame(data=d1)

#Resultados = Resultados.append(Resultados1, ignore_index=True)
Resultados1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated infavour of `both` or `neither`.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated infavour of `both` or `neither`.


132/132 [==============================] - 0s 1ms/step
[[1424  995]
 [ 399 1403]]
Precision: 0.5850708924103419
Recall: 0.7785793562708102
Accuracy: 0.6697465055674011
F1_Score: 0.6680952380952381


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:124: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated infavour of `both` or `neither`.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:127: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated infavour of `both` or `neither`.


,Modeblo,Precision,Recall,Accuracy,F1_Score
0,Red_Neuronal_1,0.585071,0.778579,0.669747,0.668095


In [6]:
import joblib

filename = 'finalized_model.sav'
joblib.dump(classifier, filename)

['finalized_model.sav']